In [1]:
import os
import pandas as pd
from tqdm.notebook import tqdm

from variables import *

### Load files

In [2]:
# Event file(s)
event_file = pd.read_csv('../data/batchfiles/batch1.gz', 
                         sep = '\t', 
                         low_memory = True,
                         engine = 'c')

# The wordID column from the regression dataframe.  
speaker_word = pd.read_csv('../data/regression_data.csv',
                           usecols=['wordID'],
                           dtype={"wordID": "category"},
                           low_memory = True, 
                           engine = 'c')

# Weights. 
df = xr.open_dataarray('../output/weights/weights_buckeye.nc')
weight_matrix = df.to_pandas()
weight_matrix = weight_matrix.transpose()
weight_matrix.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Index: 1293 entries, c.when's to y.gihn
Columns: 610 entries, please to begin
dtypes: float64(610)
memory usage: 6.1 MB


### Get activation

In [ ]:
words = speaker_word['wordID'].tolist()

df = pd.DataFrame({'activation_all': [], 'activation_segments': [], 'activation_syllables' : [], 
                   'activation_context': []})

for index, word in enumerate(speaker_word): 
    
    if index == 0:
            context = 'c.' + words[index+1]
        elif index == len(words)-1:
            context = ['c.' + words[index-1]]
        else:
            previous_word = 'c.' + words[index-1]
            following_word = 'c.' + words[index+1]
            context = [previous_word, following_word] 
    
    act = activation(word_outcome = 'the', context = [context], 
           event_files = [event_file], weight_matrix = weight_matrix, 
           domain_specific = False)
    act_domain = activation(word_outcome = 'the', context = [context], 
           event_files = [event_file], weight_matrix = weight_matrix, 
           domain_specific = True)
    
    df.at[index, 'activation_all'] = act
    df.at[index, 'activation_segments'] = act_domain['Segment']
    df.at[index, 'activation_syllables'] = act_domain['Syllable']
    df.at[index, 'activation_context'] = act_domain['Context']

ende = pd.concat(objects = [regression_data, df], axis = 1)
ende.to_csv('../data/regression_data_activation.csv', index = False)